# MPI - point to point operations

We will use `mpi4py` 

In [1]:
import numpy as np

In [5]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
print(c.ids)
view = c[:]
view.activate()

[0, 1, 2, 3, 4, 5]


Parallel eigenvalues:
```python
import numpy as np
%time np.max(np.real(np.linalg.eigvals(np.random.randn(400,400))))
```

A task: find a biggest entry in a random matrix:

In [8]:
%time np.max(np.random.randn(5000,5000))

CPU times: user 1.06 s, sys: 84 ms, total: 1.14 s
Wall time: 1.14 s


5.1695004271475078

In [4]:
%%px --block
from mpi4py import MPI
import time
import numpy as np 
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
t = MPI.Wtime()
print("result =",np.max(np.random.randn(5000,5000//4)))
t = MPI.Wtime() - t
print(rank,":: execution time:",t)

[stdout:0] 
result = 5.10738660383
0 :: execution time: 0.29744601249694824
[stdout:1] 
result = 4.96081328763
5 :: execution time: 0.29747605323791504
[stdout:2] 
result = 4.8963510546
2 :: execution time: 0.29653000831604004
[stdout:3] 
result = 4.84093788152
3 :: execution time: 0.33892011642456055
[stdout:4] 
result = 5.7277817237
4 :: execution time: 0.3216099739074707
[stdout:5] 
result = 4.99959766657
1 :: execution time: 0.32078003883361816


### Send and receive

- We use `rank` to differentiate code between processors.
- Note that mpi4py serializes arbitrary data before send.

### Important!

   -  In MPI for Python, the `Send()`, `Recv()` and `Sendrecv()`  can communicate memory buffers. 
   -  The variants `send()`, `recv()` and `sendrecv()` can communicate generic Python objects.

In [11]:
%%px --block

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = None

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)

elif rank == 1:
    data = comm.recv(source=0)

print("OK, rank= ",rank,"dane: ",data)

[stdout:0] OK, rank=  3 dane:  None
[stdout:1] OK, rank=  0 dane:  {'a': 7, 'b': 3.14}
[stdout:2] OK, rank=  1 dane:  {'a': 7, 'b': 3.14}
[stdout:3] OK, rank=  2 dane:  None


### Sending and receiving numpy arrays

- we can send the whole array

In [6]:
%%px --block
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))

if rank == 0:
    a[:] = 2
    comm.send(a, dest=1)
elif rank == 1:
    a = comm.recv(source=0)

print ("OK,",rank,np.sum(a))


[stdout:0] OK, 3 0.0
[stdout:1] OK, 2 0.0
[stdout:2] OK, 1 8.0
[stdout:3] OK, 0 8.0


 - we can send a slice !

In [6]:
%%px --block
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))

if rank == 0:
    a[:] = 2
    comm.send(a[0,:], dest=1)
elif rank == 1:
    a[0,:] = comm.recv(source=0)

print("OK,",rank,np.sum(a))


[stdout:0] OK, 0 8.0
[stdout:1] OK, 5 0.0
[stdout:2] OK, 2 0.0
[stdout:3] OK, 3 0.0
[stdout:4] OK, 4 0.0
[stdout:5] OK, 1 4.0


In [7]:
view['rank']

[0, 5, 2, 3, 4, 1]

In [11]:
view['a'][5]

array([[ 2.,  2.],
       [ 0.,  0.]])

In [15]:
np.argsort(view['rank'])

array([0, 5, 2, 3, 4, 1])

In [16]:
view['a'][np.argsort(view['rank'])[1]]

array([[ 2.,  2.],
       [ 0.,  0.]])

In [9]:
print(view['a'][view['rank'][0]])
print(view['a'][view['rank'][1]])

[[ 2.  2.]
 [ 2.  2.]]
[[ 2.  2.]
 [ 0.  0.]]


In [10]:
%%px --block

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
    a[:] = 2
    comm.send(a[:,0], dest=1)
elif rank == 1:
    a[:,0] = comm.recv(source=0)

print ("OK,",rank,np.sum(a))


[stdout:0] OK, 3 0.0
[stdout:1] OK, 2 0.0
[stdout:2] OK, 1 4.0
[stdout:3] OK, 0 8.0


In [11]:
print(view['a'][view['rank'][0]])
print(view['a'][view['rank'][1]])

[[ 2.  2.]
 [ 2.  2.]]
[[ 2.  0.]
 [ 2.  0.]]


### Communicating  memorybuffers:  `Send` and `Recv`

In [12]:
%%px --block

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
    a[:] = 2
    comm.Send(a[0,:], dest=1)
elif rank == 1:
    comm.Recv(a[0,:], source=0)

print ("OK,",rank,np.sum(a))


[stdout:0] OK, 3 0.0
[stdout:1] OK, 0 8.0
[stdout:2] OK, 1 4.0
[stdout:3] OK, 2 0.0


In [13]:
%%px --block

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
    a[:] = 2
    comm.Send(a[:,0], dest=1)
elif rank == 1:
    comm.Recv(a[:,0], source=0)

print ("OK,",rank,np.sum(a))


CompositeError: one or more exceptions from call to method: execute
[1:execute]: ValueError: ndarray is not contiguous
[2:execute]: ValueError: ndarray is not contiguous

### Contiguous memory buffers

In [14]:
a = np.zeros((2,2))
a.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [15]:
a[:,0].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [16]:
a[0,:].flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [17]:
%%px --block

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
    a[:] = 2
    buf = a[:,0].copy()
    comm.Send(buf, dest=1)   
elif rank == 1:
    buf = np.empty(2)
    comm.Recv(buf, source=0)
    a[:,0] = buf 
    print ("OK,",np.sum(a))


[stdout:2] OK, 4.0


In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
print(c.ids)
view = c[:]
view.activate()

[0, 1, 2, 3]


In [5]:
%%px --block --target :3
print("OK")

[stdout:0] OK
[stdout:1] OK
[stdout:2] OK


In [6]:
%%px --block
print("OK")

[stdout:0] OK
[stdout:1] OK
[stdout:2] OK
[stdout:3] OK


In [9]:
%%px --block
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))

if rank == 0:
    import os
    print(os.getcwd())


[stdout:3] /home/users/marcin.kostur/ProgramowanieRownolegle/MPI
